In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import h5py

## ESN

In [4]:
## ESN with bias architecture
def step(x_pre, u, sigma_in, rho):
    """ Advances one ESN time step.
        Args:
            x_pre: reservoir state
            u: input
        Returns:
            new augmented state (new state with bias_out appended)
    """
    # input is normalized and input bias added
    u_augmented = np.hstack(((u-u_mean)/norm, bias_in))
    # reservoir update
    x_post      = np.tanh(Win.dot(u_augmented*sigma_in) + W.dot(rho*x_pre))     
    # output bias added
    x_augmented = np.concatenate((x_post, bias_out))

    return x_augmented

def open_loop(U, x0, sigma_in, rho):
    """ Advances ESN in open-loop.
        Args:
            U: input time series
            x0: initial reservoir state
        Returns:
            time series of augmented reservoir states
    """
    N     = U.shape[0]
    Xa    = np.empty((N+1, N_units+1))
    Xa[0] = np.concatenate((x0,bias_out))
    for i in np.arange(1,N+1):
        Xa[i] = step(Xa[i-1,:N_units], U[i-1], sigma_in, rho)

    return Xa

def closed_loop(N, x0, Wout, sigma_in, rho):
    """ Advances ESN in closed-loop.
        Args:
            N: number of time steps
            x0: initial reservoir state
            Wout: output matrix
        Returns:
            time series of prediction
            final augmented reservoir state
    """
    xa    = x0.copy()
    Yh    = np.empty((N+1, dim))
    Yh[0] = np.dot(xa, Wout)
    for i in np.arange(1,N+1):
        xa    = step(xa[:N_units], Yh[i-1], sigma_in, rho)
        Yh[i] = np.dot(xa, Wout) #np.linalg.multi_dot([xa, Wout]) 

    return Yh, xa

def train_n(U_washout, U_train, Y_train, tikh, sigma_in, rho):
    """ Trains ESN.
        Args:
            U_washout: washout input time series
            U_train: training input time series
            tikh: Tikhonov factor
        Returns:
            time series of augmented reservoir states
            optimal output matrix
    """
        
    ## initial washout phase
    xf = open_loop(U_washout, np.zeros(N_units), sigma_in, rho)[-1,:N_units]
    
    ## splitting training in N_splits to save memory
    LHS = 0
    RHS = 0
    N_len = (U_train.shape[0]-1)//N_splits
    
    for ii in range(N_splits):
        ## open-loop train phase
        t1  = time.time()
        Xa1 = open_loop(U_train[ii*N_len:(ii+1)*N_len], xf, sigma_in, rho)[1:]
        xf  = Xa1[-1,:N_units].copy()
        if ii == 0 and k==0: print('open_loop time:', (time.time()-t1)*N_splits)
        
        ##computing the matrices for the linear system
        t1  = time.time()   
        LHS += np.dot(Xa1.T, Xa1) 
        RHS += np.dot(Xa1.T, Y_train[ii*N_len:(ii+1)*N_len])        
        if ii == 0 and k==0: print('matrix multiplication time:', (time.time()-t1)*N_splits)
    
    # to cover the last part of the data that didn't make it into the even splits
    if N_splits > 1:
        Xa1 = open_loop(U_train[(ii+1)*N_len:], xf, sigma_in, rho)[1:]
        LHS += np.dot(Xa1.T, Xa1) 
        RHS += np.dot(Xa1.T, Y_train[(ii+1)*N_len:])
    
    Wout = np.empty((len(tikh),N_units+1,dim))
    
    # solve linear system for different Tikhonov
    for j in range(len(tikh)):
        if j == 0: #add tikhonov to the diagonal (fast way that requires less memory)
            LHS.ravel()[::LHS.shape[1]+1] += tikh[j]
        else:
            LHS.ravel()[::LHS.shape[1]+1] += tikh[j] - tikh[j-1]
        
        #solve linear system
        t1  = time.time()
        Wout[j] = np.linalg.solve(LHS, RHS)

        if j==0 and k==0: print('linear system time:', (time.time() - t1)*len(tikh))

    return Wout, LHS, RHS

def train_save_n(U_washout, U_train, Y_train, tikh, sigma_in, rho, noise):
    """ Trains ESN.
        Args:
            U_washout: washout input time series
            U_train: training input time series
            tikh: Tikhonov factor
        Returns:
            time series of augmented reservoir states
            optimal output matrix
    """

    ## washout phase
    xf    = open_loop(U_washout, np.zeros(N_units), sigma_in, rho)[-1,:N_units]
    
    LHS   = 0
    RHS   = 0
    N_len = (U_train.shape[0]-1)//N_splits
    
    for ii in range(N_splits):
        t1  = time.time()
        ## open-loop train phase
        Xa1 = open_loop(U_train[ii*N_len:(ii+1)*N_len], xf, sigma_in, rho)[1:]
        xf  = Xa1[-1,:N_units].copy()

        t1  = time.time()   
        LHS += np.dot(Xa1.T, Xa1) 
        RHS += np.dot(Xa1.T, Y_train[ii*N_len:(ii+1)*N_len])
                    
    if N_splits > 1:# to cover the last part of the data that didn't make into the even splits
        Xa1 = open_loop(U_train[(ii+1)*N_len:], xf, sigma_in, rho)[1:]
        LHS += np.dot(Xa1.T, Xa1) 
        RHS += np.dot(Xa1.T, Y_train[(ii+1)*N_len:])

    LHS.ravel()[::LHS.shape[1]+1] += tikh
    
    Wout = np.linalg.solve(LHS, RHS)
    
    return Wout

## Validation

In [1]:
#Objective Functions to minimize with Bayesian Optimization

def KFC_Noise(x):
    #K-fold cross Validation
    
    global tikh_opt, k, ti
    
    #setting and initializing
    rho      = x[0]
    sigma_in = 10**x[1]
    ti       = time.time()
    lenn     = tikh.size
    Mean     = np.zeros(lenn)
    
    #Train using tv: training+val
    Wout, LHS0, RHS0 = train_n(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)
        
    #Different Folds in the validation set
    t1   = time.time()
    for i in range(N_fo):

       #select washout and validation
        p      = N_in + i*N_fw
        Y_val  = U[N_washout + p    : N_washout + p + N_val    ].copy()
        U_wash = U[            p    : N_washout + p            ].copy()
        
        #washout
        xf     = open_loop(U_wash, np.zeros(N_units), sigma_in, rho)[-1]
        #Train: remove the validation interval
        Xt     = open_loop(Y_val, xf[:N_units], sigma_in, rho)[:-1]
        
        LHS    = LHS0 - np.dot(Xt.T, Xt)
        RHS    = RHS0 - np.dot(Xt.T, Y_val)

        for j in range(lenn):
            if j == 0: #add tikhonov to the diagonal (fast way that requires less memory)
                LHS.ravel()[::LHS.shape[1]+1] += tikh[j]
            else:
                LHS.ravel()[::LHS.shape[1]+1] += tikh[j] - tikh[j-1]
            
            Wout[j]  = np.linalg.solve(LHS, RHS)

            #Validate
            Yh_val   = closed_loop(N_val-1, xf, Wout[j], sigma_in, rho)[0]
            Mean[j] += np.log10(np.mean((Y_val-Yh_val)**2))
        
                
    if k==0: print('closed-loop time:', time.time() - t1)
    
    #select optimal tikh
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    
    #print every set of hyperparameters
    if print_flag:
        print(k, ': Spectral radius, Input Scaling, Tikhonov, MSE:',
              rho, sigma_in, tikh_opt[k-1],  Mean[a]/N_fo)

    return Mean[a]/N_fo

def RVC_Noise(x):
    #Recycle Validation
    
    global tikh_opt, k, ti
    
    #setting and initializing
    rho      = x[0]
    sigma_in = 10**x[1]
    ti       = time.time()
    lenn     = tikh.size
    Mean     = np.zeros(lenn)
    
    #Train using tv: training+val
    Wout = train_n(U_washout, U_tv, Y_tv, tikh, sigma_in, rho)[0]

    #Different Folds in the validation set
    t1   = time.time()
    for i in range(N_fo):
        
        #select washout and validation
        p      = N_in + i*N_fw
        Y_val  = U[N_washout + p : N_washout + p + N_val].copy()
        U_wash = U[            p : N_washout + p        ].copy()
        
        #washout before closed loop
        xf = open_loop(U_wash, np.zeros(N_units), sigma_in, rho)[-1]
                  
        for j in range(lenn):
            #Validate
            Yh_val   = closed_loop(N_val-1, xf, Wout[j], sigma_in, rho)[0]
            Mean[j] += np.log10(np.mean((Y_val-Yh_val)**2))
                            
    if k==0: print('closed-loop time:', time.time() - t1)
    
    #select optimal tikh
    a           = np.argmin(Mean)
    tikh_opt[k] = tikh[a]
    k          +=1
    
    #print for every set of hyperparameters
    if print_flag:
        print(k, ': Spectral radius, Input Scaling, Tikhonov, MSE:',
              rho, sigma_in, tikh_opt[k-1],  Mean[a]/N_fo)

    return Mean[a]/N_fo